In [1]:
import json
import os
from moviepy.editor import *
from moviepy import *
import shutil
import numpy as np

In [2]:
# Make sure you navigate to the github repo
%cd video-classification-3d-cnn-pytorch/
os.getcwd()

C:\Users\wutai\SkillsClasses-main\SkillsClasses-main\01_Julian_VideoAndImageClassification\video-classification-3d-cnn-pytorch


'C:\\Users\\wutai\\SkillsClasses-main\\SkillsClasses-main\\01_Julian_VideoAndImageClassification\\video-classification-3d-cnn-pytorch'

In [3]:
from moviepy.editor import *

input_image_path = '../Input_films/doge.jpg'

# Read input picture
input_image = ImageClip(input_image_path)

# Gets the directory path to the input image
input_image_directory = os.path.dirname(input_image_path)

# Create a video clip lasting 2 seconds that contains 24 frames of repeated images
video = CompositeVideoClip([input_image.set_duration(4)] * 24, size=input_image.size)

# Save the video to the same directory as the input picture
video_output_path = os.path.join(input_image_directory, 'doge.mp4')
video.write_videofile(video_output_path, fps=24)


Moviepy - Building video ../Input_films\doge.mp4.
Moviepy - Writing video ../Input_films\doge.mp4



Moviepy - Done !
Moviepy - video ready ../Input_films\doge.mp4


In [4]:
# this function loads all films in the folder given as argument into the 'input' file (make sure you are in the right
# directory)

def loadVideosToInfile(folder):
    videos = os.listdir(folder)
    with open('input', 'w') as inputfile:
        for v in videos:
            if v.split('.')[-1] == 'mp4':
                inputfile.write(v+'\n')

In [5]:
# This function should read all .json files in the output folder, and combine them into one large dictionary
def readAllOutput(outputFolder):
    # create an empty dictionary called 'dictionaries' to which you add the dictionary entries read 
    # from the output files
    dictionaries = {}
    
    # Read all output file names to a variable output_files (a list that contains all file names)
    output_files = os.listdir(outputFolder)
    
    # Start of the for loop to loop through all files, read them, and add their contents to 'dictionaries'
    for f in output_files:
        # first test if the file extension is a '.json file'
        if f.endswith('.json'):    
            # if it is a json file, we open the file
            file_path = os.path.join(outputFolder, f)
            with open(file_path,'r',encoding='utf-8') as file:
                # Then load the file content to a variable called 'segments' with the json library
                segments = json.load(file)
                # another for loop, to add the content of segments to dictionaries. As a key use the moviename.
                for s in segments:
                    dictionaries[s['video']] = s
    
    return dictionaries


In [6]:
# here keyFilmName is the name of the film that you want to find a match for, and the feature dictionary is 
# the dictionary that is given as output by the readAllOutput function

def searchForMatch(keyFilmName, featureDictionary):
    # create variables to keep track of the best clips
    bestRangeStart = 0
    bestRangeFilmName = ''
    bestRangeScore = 1000000000
    
    # first read the features of the keyFilm into keyFeatures
    keyClips = featureDictionary[keyFilmName]['clips']
    keyFeatures = []
    bestMatches = []
    for c in keyClips:
        keyFeatures.append(np.array(c['features']))
    
    # now enter a for loop to go through all films, and see what the best matching sequence is 
    # (we make sure not to check the keyFilm itself by checking it's not that filename)
    for film in featureDictionary.keys():
        if not film == keyFilmName:
            
            # so now here 'film' is a filename (which is the key in the feature dictionary of that film)
            # first load the features in a 'filmFeatures' list like we did for key Features
            # then look at the provided code from the lecture to see how to iterate through the other
            # list of features to find the best segment. Now just make sure to also register what film
            # name the best matching segment is from. 
            filmClips = featureDictionary[film]['clips']
            filmFeatures = []
            filmFeatures = [np.array(s['features']) for s in filmClips]
            for i in range(len(filmFeatures)-len(keyFeatures)):
                distance = 0
                for j in range(len(keyFeatures)):
                    d = np.linalg.norm(filmFeatures[i+j]-keyFeatures[j])
                    distance += d
                if distance < bestRangeScore:
                    bestRangeScore = distance
                    bestRangeStart = i
                    bestRangeFilmName = film

                bestMatches.append({
                'bestRangeFilmName': film,
                'bestRangeStart': i,
                'bestRangeScore': distance,
                })
            
    # create a list containing the filename, start and nr of clips
    bestMatches.sort(key=lambda x: x['bestRangeScore'])
    print(bestMatches)
    bestMatches = bestMatches[0]
    
    bestRangeFilmName = bestMatches['bestRangeFilmName']
    bestRangeStart = bestMatches['bestRangeStart']
    
    matchingFragment = [bestRangeFilmName, bestRangeStart, len(keyFeatures)]
    print(matchingFragment)
    
    return matchingFragment

    # First implement it for the best matching one, then expand it to return the n best matching ones as a list,
    # with n as an argument in the function

In [7]:
# this is a helper function to which you give the fragment (the output of the searchForMatch function), and it returns
# a moviepy film object (which you can then save)

def extractFragment(fragment, filmFolder):
    # here adapt the code from the lecture, but make it so that it works for any input fragment list
    # the output of the function above: [bestRangeFilmName, bestRangeStart, len(keyFeatures)]
    
    filmFileName = fragment[0]
    start = fragment[1]*16 
    nrClips = fragment[2]*16
    
    film_path = os.path.join(filmFolder, filmFileName)
    film = VideoFileClip(film_path)
    tsfps = film.fps
    tsFrameLength = 1/tsfps
    
    startTime = start * tsFrameLength
    endTime = (start + nrClips) * tsFrameLength 
    
    filmFragment = film.subclip(startTime, endTime)
    
    return filmFragment

    # This function should read all .json files in the output folder, and combine them into one large dictionary

In [8]:
# call function to load the filenames to 'input'
loadVideosToInfile('../input_films')

In [9]:
# run the model to process all films currently in the input_films folder, and save it to a json file, here named by the date

!python main.py --input input --video_root ../input_films --output ../output/output231109.json --model resnet-34-kinetics-cpu.pth --mode feature --no_cuda

loading model resnet-34-kinetics-cpu.pth
../input_films\doge.mp4
../input_films\just-sit-down-shut-up.mp4
../input_films\KillBill.mp4


ffmpeg version 2023-11-05-git-44a0148fad-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --e

In [10]:
# a small piece of code that moves all processed input films to the processed films folder

input_films_processed = os.listdir('../input_films')
for file in input_films_processed:
    try:
        shutil.move(os.path.join('../input_films',file),'../processed_films')
    except:
        print(file + ' not moved')

In [11]:
# read all output with the function declared above

dicts = readAllOutput('../output')

In [12]:
# look for the best matching sequence to the input file (needs to be in dicts for it to work)

fragment = searchForMatch("doge.mp4", dicts)

[{'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 2, 'bestRangeScore': 106.6386886542206}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 0, 'bestRangeScore': 107.47442179606894}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 1, 'bestRangeScore': 107.56701900167562}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 25, 'bestRangeScore': 107.8216878463915}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 3, 'bestRangeScore': 108.33537687517928}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 24, 'bestRangeScore': 108.77436712417119}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 23, 'bestRangeScore': 108.81791777760672}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 22, 'bestRangeScore': 109.43497257351316}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 4, 'bestRangeScore': 110.27043786019647}, {'bestRangeFilmName': 'KillBill.mp4', 'bestRangeStart': 21, 'bestRangeScore': 111.3284095194052}, {'bestRangeFilmNa

In [13]:
# extract the fragment from the film

fragmentFilm = extractFragment(fragment, '../processed_films')

In [14]:
# display the film fragment

fragmentFilm.ipython_display()

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [15]:
# save the film fragment


fragmentFilm.write_videofile("../output_films/fragment.mp4", audio_codec='aac')

Moviepy - Building video ../output_films/fragment.mp4.
MoviePy - Writing audio in fragmentTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../output_films/fragment.mp4



Moviepy - Done !
Moviepy - video ready ../output_films/fragment.mp4
